In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, normalize_run
ensure_pyterrier_is_loaded()

import pyterrier as pt
import pandas as pd

Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
def report_effectiveness(dataset, prompt):
    queries = pt.io.read_topics(f'../{dataset}/queries.xml', 'trecxml')
    run = pd.read_json(f'{dataset}-rerank-{prompt}-with-scores.jsonl.gz', lines=True, dtype={'qid': str, 'docno': str})
    run['qid'] = run['qid'].apply(lambda i: str(i).split('_')[0])
    run = pt.Transformer.from_df(run)

    ret = pt.Experiment([run], queries, pt.io.read_qrels(f'../{dataset}/qrels.txt'), eval_metrics=["ndcg_cut_10", 'recip_rank', 'P_10', 'recall_100', 'recall_1000', 'recall_5000'])
    
    return {i: ret.iloc[0][i] for i in ['ndcg_cut_10', 'recip_rank', 'P_10', 'recall_100', 'recall_1000', 'recall_5000']}

def report_pipeline():
    ret = []

    for prompt in ['orig', '1', '2', '3', '4']:
        i = {'Prompt': prompt}
        for dataset in ['tot-train', 'tot-dev']:
            for k,v in report_effectiveness(dataset, prompt).items():
                i[dataset.split('-')[1] + '_' + k] = v
        ret += [i]
    return pd.DataFrame(ret)

report_pipeline()

,Prompt,train_ndcg_cut_10,train_recip_rank,train_P_10,train_recall_100,train_recall_1000,train_recall_5000,dev_ndcg_cut_10,dev_recip_rank,dev_P_10,dev_recall_100,dev_recall_1000,dev_recall_5000
0,orig,0.030571,0.029204,0.006000,0.246667,0.626667,0.626667,0.046061,0.045860,0.007333,0.260000,0.566667,0.566667
1,1,0.119979,0.104267,0.019333,0.413333,0.626667,0.626667,0.145610,0.135857,0.020000,0.380000,0.566667,0.566667
2,2,0.107167,0.093127,0.018667,0.420000,0.626667,0.626667,0.112063,0.096276,0.018667,0.360000,0.566667,0.566667
3,3,0.089249,0.076313,0.015333,0.326667,0.626667,0.626667,0.090465,0.075247,0.016667,0.360000,0.566667,0.566667
4,4,0.106904,0.098330,0.016667,0.426667,0.626667,0.626667,0.112698,0.102139,0.017333,0.373333,0.566667,0.566667


### Fusion Approaches

In [6]:
!pip3 install ranx

from ranx import Run, fuse

     |████████████████████████████████| 95 kB 911 kB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 190 kB 20.0 MB/s eta 0:00:01
     |████████████████████████████████| 239 kB 23.7 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 19.0 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 34.6 MB 196 kB/s eta 0:00:011
     |████████████████████████████████| 1.2 MB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 1.0 MB/s  eta 0:00:01
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2
